<a href="https://colab.research.google.com/github/zogojogo/Food-Detection-w-Machine-Learning/blob/main/Food_Detection_w_Food101_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Ambil data dari kaggle food101
2. Tambah Data foto makanan Indo
3. Pre-process
4. Image Annotation (buat faster R-CNN)
5. Buat modelnya
6. Train
7. Predict



##Setting Kaggle

In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kriswiliant0","key":"921c9a4cd135e9afa02eeb07e5d78126"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

##Donlot Dataset

In [3]:
! kaggle datasets download -d kmader/food41

100% 5.30G/5.30G [01:14<00:00, 45.2MB/s]
100% 5.30G/5.30G [01:14<00:00, 76.2MB/s]


In [4]:
import os
import zipfile

local_zip = '/content/food41.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

In [6]:
!mkdir /content/testset
fl = open('/content/meta/meta/classes.txt')
cls = fl.readline()
while(cls):
    !mkdir /content/testset/{cls}
    cls = fl.readline()
# Moving test files to testset/, train files will be left.
testfile = open('/content/meta/meta/test.txt')
img = testfile.readline().strip()
while(img):
    cls = img.split('/')[0]
    !mv /content/images/{img}.jpg /content/testset/{cls}/
    print(f'\r{img}',end='')
    img = testfile.readline().strip()

waffles/971843

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        shear_range=0.3,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.25,
)
valid_datagen = ImageDataGenerator(
        rescale=1./255,
)

In [3]:
batch_size = 128

train_data = train_datagen.flow_from_directory(
    '/content/images',
    batch_size=batch_size,
    target_size=(224, 224),
    shuffle=True,
    # class_mode = 'categorical'
)
valid_data = valid_datagen.flow_from_directory(
    '/content/testset/',
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=False,
    # class_mode = 'categorical'
)


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [4]:
base = tf.keras.applications.resnet.ResNet50(input_shape=(224,224,3),include_top=False,weights='imagenet')
base.trainable = True
model = Sequential()
model.add(base)
# model.add(Flatten())
model.add(GlobalAveragePooling2D())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss = 'categorical_crossentropy',metrics=['accuracy'])

94773248/94765736 [==============================] - 1s 0us/step


In [5]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 1,verbose = 1)
early_stop = EarlyStopping(monitor = 'val_accuracy',patience = 5,verbose = 1,restore_best_weights = True)

In [ ]:
history = model.fit(train_data, 
                    epochs=10,
                    validation_data = valid_data, 
                    verbose = 1,
                    callbacks=[early_stop, reduce_lr])

In [2]:
!git commit

fatal: not a git repository (or any of the parent directories): .git
